In [1]:
# Install packages if not present

!pip install coiled==0.2.16 dask==2022.12.0 itk==5.3.0 vtk git+https://github.com/uncbiag/OAI_analysis_2.git

  Cloning https://github.com/uncbiag/OAI_analysis_2.git to /tmp/pip-req-build-8z1x53sv
  Running command git clone --filter=blob:none --quiet https://github.com/uncbiag/OAI_analysis_2.git /tmp/pip-req-build-8z1x53sv
  Resolved https://github.com/uncbiag/OAI_analysis_2.git to commit fb1abd8972896d1da31b589aa560bd2cca55853f
  Preparing metadata (setup.py) ... done
  Using cached coiled-0.2.16-py3-none-any.whl (139 kB)
  Using cached dask-2022.12.0-py3-none-any.whl (1.1 MB)
  Using cached itk-5.3.0-cp38-cp38-manylinux_2_28_x86_64.whl (8.3 kB)
  Using cached vtk-9.2.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (79.3 MB)
  Using cached rich-13.3.1-py3-none-any.whl (239 kB)
  Using cached textual-0.10.1-py3-none-any.whl (298 kB)
  Using cached backoff-1.11.1-py2.py3-none-any.whl (13 kB)
  Using cached aiohttp-3.8.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached boto3-1.26.62-py3-none-any.whl (132 kB)
  Using cached aiobotocore-2.4.2-py3-none-an

In [2]:
# All Imports

import coiled
import dask
from dask import compute, visualize, delayed

import time
import random
import itk
import vtk
import oai_analysis_2
from oai_analysis_2 import dask_processing as dp

In [8]:
# Start Coiled Cluster
# Enter Token when asked

import coiled
import dask
from dask import compute, visualize, delayed

cluster = coiled.Cluster(
    name='gpucluster',
    scheduler_vm_types=['t3.medium'],
    worker_vm_types=["g4dn.2xlarge", "p3.2xlarge", "g5.2xlarge"],
    n_workers=6,
    software="thewtex/oaiimageanalysis",
    worker_options={"nthreads": 1},
    shutdown_on_close=True,
)

client = dask.distributed.Client(cluster)

╭───────────────────────────────────────── Coiled Cluster ─────────────────────────────────────────╮
│                     https://cloud.coiled.io/thewtex/clusters/152060/details                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────── Overview ───────────────────╮╭──────────────── Configuration ─────────────────╮
│                                                ││                                                │
│ Cluster Name: gpucluster                       ││ Region: us-east-1                              │
│                                                ││                                                │
│ Cluster Status: ready                          ││ Scheduler Instance Type: t3.medium             │
│                                                ││                                                │
│ Scheduler Status: started                      ││ Worker Instance Type(s): g4dn.2xlarge (2),     │
│                                                ││ p3.2xlarge (4)                                 │
│ Dashboard Address: ]8;id=562430;http://52.90.225.34:8787\http://52.90.225.34:8787]8;;\    ││                                                │
│                                                ││ Workers Requested: 6                           │
│                                                ││                                                │
╰────────────────────────────────────────────────╯╰────────────────────────────────────────────────╯
╭─────────────────────────────────── (2023/02/02 11:57:38 EST) ────────────────────────────────────╮
│                                                                                                  │
│                                        All workers ready.                                        │
│                                                                                                  │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯


╭───────────────────────────────────────── Coiled Cluster ─────────────────────────────────────────╮
│                     https://cloud.coiled.io/thewtex/clusters/152060/details                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────── Overview ───────────────────╮╭──────────────── Configuration ─────────────────╮
│                                                ││                                                │
│ Cluster Name: gpucluster                       ││ Region: us-east-1                              │
│                                                ││                                                │
│ Cluster Status: ready                          ││ Scheduler Instance Type: t3.medium             │
│                                                ││                                                │
│ Scheduler Status: started                      ││ Worker Instance Type(s): g4dn.2xlarge (2),     │
│                                                ││ p3.2xlarge (4)                                 │
│ Dashboard Address: ]8;id=940848;http://52.90.225.34:8787\http://52.90.225.34:8787]8;;\    ││                                                │
│                                                ││ Workers Requested: 6                           │
│                                                ││                                                │
╰────────────────────────────────────────────────╯╰────────────────────────────────────────────────╯
╭─────────────────────────────────── (2023/02/02 11:57:38 EST) ────────────────────────────────────╮
│                                                                                                  │
│                                        All workers ready.       

In [10]:
# Main execution loop
# Change the patients_array and months_array based on requirements.
# Following code will run the computation on all samples.

atlas_path = "s3://oaisample1/ZARRDATA/atlas_image.zarr/scale0"
write_path = "/home/matt/data/OAIFULLDATA/RESULTS1/"

patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]

counter = 0

print(time.time())
random.shuffle(patients_array)

for patient_id in patients_array:
    import time
    print('Processing ', patient_id, time.time())
    patient_result = []
    for month in months_array:
        image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_0.zarr/scale0'
        
        fc_probmap, tc_probmap = dp.segment_method(image_path)
        phi_AB, image_A, image_B = dp.register_images_delayed(image_path, atlas_path)

        deformed_fc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
                                             fc_probmap, 'FC')
        deformed_tc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
                                             tc_probmap, 'TC')
        thickness_fc = dp.get_thickness(deformed_fc, 'FC')
        thickness_tc = dp.get_thickness(deformed_tc, 'TC')
        
        patient_result.append([thickness_fc, thickness_tc])
    
    output_result = compute(*patient_result)
    for result in output_result:
        itk.meshwrite(
            result[0], write_path + patient_id + '_' +
            str(counter) + '_itk_distance_inner_FC.vtk')
        itk.meshwrite(
            result[1], write_path + patient_id + '_' +
            str(counter) + '_itk_distance_inner_TC.vtk')
        counter = counter + 1
    
print(time.time())

1675357595.918026
Processing  9247140 1675357595.9181752


2023-02-02 12:08:56,585 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/protocol/core.py", line 158, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/protocol/core.py", line 137, in _decode_default
    sub_frames = decompress(sub_header, sub_frames)
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/protocol/compression.py", line 197, in decompress
    return [
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/protocol/compression.py", line 198, in <listcomp>
    compressions[c]["decompress"](frame)
KeyError: 'lz4'


KeyError: 'lz4'

2023-02-02 12:43:08,341 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2023-02-02 12:45:21,366 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/comm/core.py", line 291, in connect
    comm = await asyncio.wait_for(
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/asyncio/tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 155, in _sync_cluster_info
    await self.scheduler_comm.set_metadata(
  File "/home/matt/bin/mambaforge/envs/oai-python38/lib/

In [7]:
# Close the cluster after computation

cluster.close()